In [ ]:
for surgery in ['hip', 'knee'] :
    for yr in ['1', '2', '3', '0.5', '0.25'] :
        dataset = surgery + '_' + yr + 'yr'
        %run "Discovery Experiments"/run_models.py --dataset {dataset} --data_dir=. --output_dir='outputs/' \
        --exp_types lr lda vanilla attention --structured --display --bsize 8 --n_iters=20

In [ ]:
for surgery in ['hip', 'knee'] :
    for yr in ['1', '2', '3', '0.5', '0.25'] :
        dataset = surgery + '_' + yr
        %run "Discovery Experiments"/run_models.py --dataset {dataset} --data_dir=. --output_dir='outputs/' \
        --exp_types vanilla attention --display --bsize 8 --n_iters=20

In [1]:
from dataloaders import dataloaders

In [7]:
import argparse
parser = argparse.ArgumentParser(description='Run experiments on a dataset')
parser.add_argument("--data_dir", type=str, required=True)

args, extras = parser.parse_known_args(['--data_dir=.'])
args.extras = extras

dataset = dataloaders['both_1yr'](args)

INFO - 2019-04-13 15:14:54,032 - Reading Structured data ...
INFO - 2019-04-13 15:14:54,809 - Reading Notes ...
INFO - 2019-04-13 15:14:57,303 - Stratifying ...


In [8]:
train_data = dataset.get_data('train', structured=False)

Pos Percentage [0.04390465 0.44930597 0.05099578 0.55069403]


In [9]:
from Experiments.modifiable_config_exp import LSTM

In [11]:
config = LSTM(dataset, True, None)

In [12]:
from models.Vanilla import ClassificationTrainer
trainer = ClassificationTrainer(config)

ModuleNotFoundError: No module named 'Attention'

In [ ]:
import numpy as np
import torch
target = np.array(train_data.y)
np.array(train_data.y).reshape(-1, 2, 2)